## Task 5&6 
Our own implementation for the street view data.

In [ ]:
# importing the packages and loading the data
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from numpy.random import choice, random_integers
from data import Data
import tensorflow as tf

data = Data()
img, label = data.get_batch()

In [ ]:
# plotting examples
print(label[5])
print(img[5].shape)

plt.title("Test image with label %s"%(label[5]))
plt.imshow(np.squeeze(img[1]), cmap = plt.get_cmap('gray'), vmin=0, vmax=255)
plt.show()

In [ ]:
# first throw: use net from last week
x = tf.placeholder(tf.float32, (None, 28, 28, 1), name='input')
t = tf.placeholder(tf.int64, (None), name='target')

wght_i = tf.Variable(tf.truncated_normal((5,5,1,32), stddev=0.1), name='kernel')
bias_i = tf.Variable(tf.constant(0.1, shape=(32,)), name='biases')
conv_i = tf.nn.conv2d(x, wght_i, (1,1,1,1), 'SAME', name='convolution')
acti_i = tf.tanh(conv_i + bias_i, name='activation')
pool_i = tf.nn.max_pool(acti_i, (1,2,2,1), (1,2,2,1), 'SAME', name='pool')

wght_h = tf.Variable(tf.truncated_normal((5,5,32,64), stddev=0.1), name='kernel')
bias_h = tf.Variable(tf.constant(0.1, shape=(64,)), name='biases')
conv_h = tf.nn.conv2d(pool_i, wght_h, (1,1,1,1), 'SAME', name='convolution')
acti_h = tf.tanh(conv_h + bias_h, name='activation')
pool_h = tf.nn.max_pool(acti_h, (1,2,2,1), (1,2,2,1), 'SAME', name='pool')


wght_oh = tf.Variable(tf.random_normal((7*7*64,1024)), name='readout_hidden_weights')
bias_oh = tf.Variable(tf.zeros((1024,)), name='readout_hidden_biases')
read_oh = tf.nn.relu(tf.matmul(tf.reshape(pool_h, (-1,7*7*64)), wght_oh) + bias_oh, name='readout_hidden')

wght_o = tf.Variable(tf.random_normal((1024,10)), name='weights')
bias_o = tf.Variable(tf.zeros((10,)), name='biases')
y = tf.matmul(read_oh, wght_o) + bias_o


entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(y, t), name='entropy')
train_step = tf.train.AdamOptimizer(1e-4).minimize(entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), t)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# Training of the model
sess = tf.Session()
sess.run(tf.initialize_all_variables())

EPOCHS = 1000

plt.ion()
figs, ax = plt.subplots(1, 1)
plt.title('Training...')
acc = []

for i in range(1, EPOCHS + 1):
    batch_x, batch_t = data.get_batch(10, src=data.test)
    sess.run(train_step, {x: batch_x, t: batch_t})
    if i % 100 == 0:
        acc_b, acc_t= data.get_batch(1000, src=data.test)
        acc_val = sess.run(accuracy, {x: acc_b, t: acc_t})
        acc.append(acc_val)
        print(acc_val)


In [ ]:
# plotting all the things
plt.plot(acc,'*')